Импорт библиотек

In [1]:
import pandas as pd
import psycopg2
from datetime import datetime, timedelta

import it4fin_functions as it4fin # мои функции выделенные в отдельный модуль

In [2]:
# настройка для pandas для отображения всех строк и колонок
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Создание соединения с базой данных

In [3]:
cnxn = psycopg2.connect(user='postgres',
                        database='postgres',
                        host='localhost',
                        port='5432',
                        password='12345')
print(cnxn.get_dsn_parameters())

# проверочный запрос
pd.read_sql_query("SELECT * FROM stock_orders LIMIT 1", cnxn)

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


,no,security_code,buysell,order_no,action,price,volume,trade_no,trade_price,order_time
0,36431,RSTI,S,23291,1,0.4562,40000,None,None,10:00:24.718000


In [4]:
def get_spread_series(cnxn, security_code, end_time, start_time=None, freq=None):
    '''
    Основная функция для выполнения Д/З

    P.S. Для больших интервалов считает долго, поэтому лучше не задавать интервал 2 часа и частоту 1000ms - будет считать около часа
    Можно еще через pandas делать, вырезая сначала таблицу между для основного временного интервала,
    но я боялся потенциальной ошибки OUT OF MEMORY

    :param cnxn: соединение с базой данных
    :param security_code: тикер для ценной бумаги, например, 'AFLT'
    :param start_time: (optional) начало временного интервала, в формате 'HH:MI:SS.mss'
    :param end_time: конец временного интервала в том же формате
    :param freq: (optional), частота в миллисекундах

    :returns: `DataFrame`, определяющий биржевой стакан для ценной бумаги (buy volume - price - sell volume) и бид аск спред как `int` в слyчае заданы только `end_time` и `security_code` . Если, в добавок, даны `start_time` и `freq` то так же возвращает `Series` бид-аск спредов для временного интервала между `start_time` и `end_time`

    '''

    # считываем шаблон запроса для получения таблицы
    with open('template_get_remaining_volumes.sql', 'r', encoding='utf-8') as f:
        get_remaining_volumes = f.read()

    # печатаем полученные на вход аргументы
    print(security_code, start_time, end_time)

    # дальше ветка: если нет start_time или частота нулевая
    if start_time is None or freq is None or freq == 0:
        # то формируем запрос на получение таблицы заявок до времени end_time и выполняем его
        query = get_remaining_volumes.format(security_code = security_code,
                                             time1 = '0:00:00',
                                             time2 = end_time)
        raw = pd.read_sql_query(query, cnxn)

        # превращаем эту таблицу в стакан
        dom, bid, ask = it4fin.get_dom_from_raw(raw)
        # возвращаем бид-аск спред и стакан
        return (ask - bid) * 2 / (ask + bid), dom

    # теперь когда заданы все аргументы
    # проверяем то как задан формат времени
    if '.' in start_time:
        stime = datetime.strptime(start_time, '%H:%M:%S.%f')
    else:
        stime = datetime.strptime(start_time, '%H:%M:%S')

    if '.' in end_time:
        etime = datetime.strptime(end_time, '%H:%M:%S.%f')
    else:
        etime = datetime.strptime(end_time, '%H:%M:%S')

    # создаём массивы для времени и спредов
    times, spreads = [], []

    query = get_remaining_volumes.format(security_code = security_code,
                                         time1 = '0:00:00',
                                         time2 = start_time)
    new_raw = pd.read_sql_query(query, cnxn)
    dom, bid, ask = it4fin.get_dom_from_raw(new_raw)

    # добавляем в массивы координаты
    times.append(stime.strftime('%H:%M:%S.%f'))
    spreads.append((ask - bid) * 2 / (ask + bid))

    while True:
        time1 = stime
        time2 = stime + timedelta(milliseconds=freq)
        if time2 > etime:
            time2 = etime

        # print(time1.strftime('%H:%M:%S.%f'), time2.strftime('%H:%M:%S.%f'))
        query = get_remaining_volumes.format(security_code = security_code,
                                             time1 = time1.strftime('%H:%M:%S.%f'),
                                             time2 = time2.strftime('%H:%M:%S.%f'))

        raw1 = new_raw
        raw2 = pd.read_sql_query(query, cnxn)

        if not raw2.empty:
            new_raw = pd.concat([raw1, raw2], axis=0, ignore_index=True).\
                groupby(by=["buysell", "order_no", "price"], as_index=False).sum()

            dom, bid, ask = it4fin.get_dom_from_raw(new_raw)
            spreads.append((ask - bid) * 2 / (ask + bid))
        else:
            new_raw = raw1
            spreads.append(spreads[-1])

        stime = time2
        times.append(stime.strftime('%H:%M:%S.%f'))

        if stime == etime:
            return pd.Series(data=spreads, index=times), dom

In [5]:
%%time
# чекаем случай без спреда
bds, dom = get_spread_series(cnxn,
                             security_code='YNDX',
                             end_time='13:00:00')
display(bds)
display(dom)
print(dom.shape)

YNDX None 13:00:00


0.0014926028359454285

,buy,sell
price,,
1222.9,0.0,100.0
1190.3,0.0,2.0
1189.0,0.0,2.0
1173.6,0.0,10.0
1160.0,0.0,50.0
1156.2,0.0,3.0
1152.7,0.0,2.0
1150.5,0.0,1.0
1150.0,0.0,731.0


(83, 2)
Wall time: 2.77 s


In [6]:
%%time
# чекаем случай со спердом
bds, dom = get_spread_series(cnxn,
                             security_code='YNDX',
                             start_time='12:00:00',
                             end_time='13:00:00',
                             freq=1000*60) # 1 минута
display(bds)
display(dom)
print(dom.shape)

YNDX 12:00:00 13:00:00


KeyboardInterrupt: 

In [7]:
%%time
# случай для особо умных, которые решили задать время и нулевую частоту
bds, dom = get_spread_series(cnxn,
                  security_code='YNDX',
                  start_time='12:00:00',
                  end_time='13:00:00',
                  freq=0)
display(bds)
display(dom)
print(dom.shape)

YNDX 12:00:00 13:00:00


0.0014926028359454285

,buy,sell
price,,
1222.9,0.0,100.0
1190.3,0.0,2.0
1189.0,0.0,2.0
1173.6,0.0,10.0
1160.0,0.0,50.0
1156.2,0.0,3.0
1152.7,0.0,2.0
1150.5,0.0,1.0
1150.0,0.0,731.0


(83, 2)
Wall time: 1.92 s


In [ ]:
%%time
# чекаем случай со спердом
bds, dom = get_spread_series(cnxn,
                             security_code='YNDX',
                             start_time='12:00:00',
                             end_time='14:00:00',
                             freq=1000*60*15) # 15 минут
display(bds)
# display(dom)
# print(dom.shape)

YNDX 12:00:00 14:00:00
